# Process for NFL Data

In [2]:
import urllib2
import pandas as pd
import numpy as np
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
opener = urllib2.build_opener()
opener.addheaders = [('User-Agent', 'Mozilla/5.0')]

#ADBLOCKERS FOR CHROME DRIVER
path_ = '../ADBLOCKERS/2.26_0'
options = webdriver.ChromeOptions()
options.add_extension('../ADBLOCKERS/adblock-plus-crx-master/bin/Adblock-Plus_v1.12.4.crx')
options.add_argument('load-extension=' + path_)

### 1. Get the Ingredients 

#### Main Dataframe Starts Here

In [3]:
df = pd.read_csv('season_2017.csv')

In [4]:
df.head()

,Unnamed: 0,Week,Home_team,Away_team,Vegas_line,Over_under,Day,Gametime,Home_points,Away_points,...,Away_line,Away_cover,Home_win,Away_win,home_pct_games_covered,away_pct_games_covered,home_pct_games_won,away_pct_games_won,home_pct_games_over,away_pct_games_over
0,0,1,NWE,KAN,8.0,1,Thu,8:30PM,27,42,...,8.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,CHI,ATL,6.5,0,Sun,1:00PM,17,23,...,-6.5,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1,BUF,NYJ,7.0,0,Sun,1:00PM,21,12,...,7.0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1,CLE,PIT,10.0,0,Sun,1:00PM,18,21,...,-10.0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1,CIN,BAL,2.5,0,Sun,1:00PM,0,20,...,2.5,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Cook up the Averages

#### Passing Stats

In [5]:
def pass_complete_pct(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    pass_att    = 0
    pass_complt = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            pass_att    += df_mini['home_pass_att'][i]
            pass_complt += df_mini['home_pass_complt'][i]
        if df_mini['Away_team'][i] == team:
            pass_att    += df_mini['vis_pass_att'][i]
            pass_complt += df_mini['vis_pass_complt'][i]
        try:
            complete_pct = (pass_complt / pass_att) * 100
            ##TROUBLESHOOTING HERE
            print pass_complt, pass_att
        except:
            complete_pct = 0
        return complete_pct
    
def pass_complete_pct_def(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    pass_att    = 0
    pass_complt = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            pass_att    += df_mini['vis_pass_att'][i]
            pass_complt += df_mini['vis_pass_complt'][i]
        if df_mini['Away_team'][i] == team:
            pass_att    += df_mini['home_pass_att'][i]
            pass_complt += df_mini['home_pass_complt'][i]
        try:
            complete_pct = (pass_complt / pass_att) * 100
        except:
            complete_pct = 0
        return complete_pct
    
def deep_complete_pct(x):
    team = x[0]
    week = x[1]

    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    deep_att    = 0
    deep_complt = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            deep_att    += df_mini['home_deep_att'][i]
            deep_complt += df_mini['home_deep_complt'][i]
        if df_mini['Away_team'][i] == team:
            deep_att    += df_mini['vis_deep_att'][i]
            deep_complt += df_mini['vis_deep_complt'][i]
        try:
            deep_complt_pct = (deep_complt / deep_att) * 100
            ##TROUBLESHOOTING HERE
#           print deep_att,deep_complt
        except:
            deep_complt_pct = 0
        return deep_complt_pct

def deep_complete_pct_def(x):
    team = x[0]
    week = x[1]

    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    deep_att    = 0
    deep_complt = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            deep_att    += df_mini['vis_deep_att'][i]
            deep_complt += df_mini['vis_deep_complt'][i]
        if df_mini['Away_team'][i] == team:
            deep_att    += df_mini['home_deep_att'][i]
            deep_complt += df_mini['home_deep_complt'][i]
        try:
            deep_complt_pct = (deep_complt / deep_att) * 100
        except:
            deep_complt_pct = 0
        return deep_complt_pct
    
def yards_per_pass(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    ypp_avg = 0
    pass_plays = 0
    pass_yards = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            pass_plays += df_mini['home_pass_att'][i]
            pass_yards += df_mini['home_pass_yds'][i]
        elif df_mini['Away_team'][i] == team:
            pass_plays += df_mini['vis_pass_att'][i]
            pass_yards += df_mini['vis_pass_yds'][i]
    try:
        ypp_avg = pass_yards / pass_plays
    except:
        ypp_avg = 0
    return ypp_avg

def yards_per_pass_def(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    ypp_avg = 0
    pass_plays = 0
    pass_yards = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            pass_plays += df_mini['vis_pass_att'][i]
            pass_yards += df_mini['vis_pass_yds'][i]
        elif df_mini['Away_team'][i] == team:
            pass_plays += df_mini['home_pass_att'][i]
            pass_yards += df_mini['home_pass_yds'][i]
    try:
        ypp_avg = pass_yards / pass_plays
    except:
        ypp_avg = 0
    return ypp_avg

def pass_yards_per_game(x):
    team = x[0]
    week = x[1]

    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    total_pass = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            total_pass += df_mini['home_pass_yds'][i]
        elif df_mini['Away_team'][i] == team:
            total_pass += df_mini['vis_pass_yds'][i]
    try:
        pass_yds_per_game = total_pass / len(df_mini)        
    except:
        pass_yds_per_game = 0
    return pass_yds_per_game  

def pass_yards_per_game_def(x):
    team = x[0]
    week = x[1]

    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    total_pass = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            total_pass += df_mini['vis_pass_yds'][i]
        elif df_mini['Away_team'][i] == team:
            total_pass += df_mini['home_pass_yds'][i]
    try:
        pass_yds_per_game = total_pass / len(df_mini)        
    except:
        pass_yds_per_game = 0
    return pass_yds_per_game  

def interception_pct(x):
    team = x[0]
    week = x[1]

    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    pass_att = 0
    interceptions_thrown = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            pass_att             += df_mini['home_pass_att'][i]
            interceptions_thrown += df_mini['home_interceptions_thrown'][i]
        if df_mini['Away_team'][i] == team:
            pass_att             += df_mini['vis_pass_att'][i]
            interceptions_thrown += df_mini['vis_interceptions_thrown'][i]
    try:
        interception_pct = (interceptions_thrown / pass_att) * 100
    except:
        interception_pct = 0
    return interception_pct

def interception_pct_def(x):
    team = x[0]
    week = x[1]

    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    pass_att = 0
    interceptions_thrown = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            pass_att             += df_mini['vis_pass_att'][i]
            interceptions_thrown += df_mini['vis_interceptions_thrown'][i]
        if df_mini['Away_team'][i] == team:
            pass_att             += df_mini['home_pass_att'][i]
            interceptions_thrown += df_mini['home_interceptions_thrown'][i]
    try:
        interception_pct = (interceptions_thrown / pass_att) * 100
    except:
        interception_pct = 0
    return interception_pct

def sack_pct(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    pass_att = 0
    total_sacks = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            pass_att    += df_mini['home_pass_att'][i]
            total_sacks += df_mini['home_sacked'][i]
        if df_mini['Away_team'][i] == team:
            pass_att    += df_mini['vis_pass_att'][i]
            total_sacks += df_mini['vis_sacked'][i]
    try:
        sack_pct = (total_sacks / pass_att) * 100
    except:
        sack_pct = 0
    return sack_pct

def sack_pct_def(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    pass_att = 0
    total_sacks = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            pass_att    += df_mini['vis_pass_att'][i]
            total_sacks += df_mini['vis_sacked'][i]
        if df_mini['Away_team'][i] == team:
            pass_att    += df_mini['home_pass_att'][i]
            total_sacks += df_mini['home_sacked'][i]
    try:
        sack_pct = (total_sacks / pass_att) * 100
    except:
        sack_pct = 0
    return sack_pct

In [6]:
df['home_pass_complete_pct']         =  df[['Home_team','Week']].apply(pass_complete_pct,axis=1)
df['home_pass_complete_pct_def']     =  df[['Home_team','Week']].apply(pass_complete_pct_def,axis=1)
df['away_pass_complete_pct']         =  df[['Away_team','Week']].apply(pass_complete_pct,axis=1)
df['away_pass_complete_pct_def']     =  df[['Away_team','Week']].apply(pass_complete_pct_def,axis=1)

df['home_deep_complete_pct']         =  df[['Home_team','Week']].apply(deep_complete_pct,axis=1)
df['home_deep_complete_pct_def']     =  df[['Home_team','Week']].apply(deep_complete_pct_def,axis=1)
df['away_deep_complete_pct']         =  df[['Away_team','Week']].apply(deep_complete_pct,axis=1)
df['away_deep_complete_pct_def']     =  df[['Away_team','Week']].apply(deep_complete_pct_def,axis=1)

df['home_yds_per_pass']              =  df[['Home_team','Week']].apply(yards_per_pass,axis=1)
df['home_yds_per_pass_def']          =  df[['Home_team','Week']].apply(yards_per_pass_def,axis=1)
df['away_yds_per_pass']              =  df[['Away_team','Week']].apply(yards_per_pass,axis=1)
df['away_yds_per_pass_def']          =  df[['Away_team','Week']].apply(yards_per_pass_def,axis=1)

df['home_pass_yds_pg']               =  df[['Home_team','Week']].apply(pass_yards_per_game,axis=1)
df['home_pass_yds_pg_def']           =  df[['Home_team','Week']].apply(pass_yards_per_game_def,axis=1)
df['away_pass_yds_pg']               =  df[['Away_team','Week']].apply(pass_yards_per_game,axis=1)
df['away_pass_yds_pg_def']           =  df[['Away_team','Week']].apply(pass_yards_per_game_def,axis=1)

df['home_interception_pct']          = df[['Home_team','Week']].apply(interception_pct,axis=1)
df['home_interception_pct_def']      = df[['Home_team','Week']].apply(interception_pct_def,axis=1)
df['away_interception_pct']          = df[['Away_team','Week']].apply(interception_pct,axis=1)
df['away_interception_pct_def']      = df[['Away_team','Week']].apply(interception_pct_def,axis=1)

df['home_sack_pct']                  = df[['Home_team','Week']].apply(sack_pct,axis=1)
df['home_sack_pct_def']              = df[['Home_team','Week']].apply(sack_pct_def,axis=1)
df['away_sack_pct']                  = df[['Away_team','Week']].apply(sack_pct,axis=1)
df['away_sack_pct_def']              = df[['Away_team','Week']].apply(sack_pct_def,axis=1)

16 31
28 35
11 21
14 25
27 37
24 36
9 17
11 21
22 33
22 32
17 28
22 30
14 27
21 30
29 38
24 35
11 21
26 39
16 36
26 39
29 41
16 28
27 32
26 40
14 25
11 21
25 41
28 42
22 33
23 40
27 48
28 42
24 34
16 36
19 36
20 30
26 39
24 39
27 32
9 17
21 30
27 48
18 31
22 33
17 28
14 27
28 35
18 31
24 34
20 30
29 41
26 39
24 36
29 38
11 21
16 31
22 32
22 30
24 39
19 36
26 40
14 25
21 30
27 32
26 39
19 36
9 17
27 37
23 40
27 48
11 21
28 35
22 32
17 28
25 41
22 32
16 28
24 34
22 30
28 42
11 21
20 30
27 32
26 40
24 35
24 36
22 33
29 38
16 36
26 39
9 17
20 30
16 28
26 39
16 31
18 31
26 39
16 36
27 37
14 27
23 40
29 41
28 35
26 39
11 21
26 39
29 38
25 41
27 37
14 25
19 36
14 27
24 35
24 39
24 34
28 42
27 48
16 28
26 40
11 21
23 40
29 41
25 41
11 21
18 31
22 30
21 30
24 35
17 28
14 25
24 36
26 40
27 32
29 38
28 42
20 30
27 37
19 36
24 34
22 33
17 28
22 32
24 39
14 27
29 41
24 39
23 40
11 21
28 35
26 39
21 30
26 39
16 31
16 36
24 35
22 30
27 48
22 32
24 36
9 17
24 39
26 40
21 30
24 34
28 42
11 21
25 41
16 

#### Rushing Stats

In [514]:
def positive_run_pct(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    rush_att     = 0 
    positive_run = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            rush_att     += df_mini['home_rush_att'][i]
            positive_run += df_mini['home_positive_run'][i]
        if df_mini['Away_team'][i] == team:
            rush_att     += df_mini['vis_rush_att'][i]
            positive_run += df_mini['vis_positive_run'][i]
    try:
        positive_run_pct = (positive_run / rush_att) * 100
    except:
        positive_run_pct = 0
    return positive_run_pct

def positive_run_pct_def(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    rush_att     = 0 
    positive_run = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            rush_att     += df_mini['vis_rush_att'][i]
            positive_run += df_mini['vis_positive_run'][i]
        if df_mini['Away_team'][i] == team:
            rush_att     += df_mini['home_rush_att'][i]
            positive_run += df_mini['home_positive_run'][i]
    try:
        positive_run_pct = (positive_run / rush_att) * 100
    except:
        positive_run_pct = 0
    return positive_run_pct
    
def negative_run_pct(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    rush_att     = 0 
    negative_run = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            rush_att     += df_mini['home_rush_att'][i]
            negative_run += df_mini['home_negative_run'][i]
        if df_mini['Away_team'][i] == team:
            rush_att     += df_mini['vis_rush_att'][i]
            negative_run += df_mini['vis_negative_run'][i]
    try:
        negative_run_pct = (negative_run / rush_att) * 100
    except:
        negative_run_pct = 0
    return negative_run_pct  

def negative_run_pct_def(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    rush_att     = 0 
    negative_run = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            rush_att     += df_mini['vis_rush_att'][i]
            negative_run += df_mini['vis_negative_run'][i]
        if df_mini['Away_team'][i] == team:
            rush_att     += df_mini['home_rush_att'][i]
            negative_run += df_mini['home_negative_run'][i]
    try:
        negative_run_pct = (negative_run / rush_att) * 100
    except:
        negative_run_pct = 0
    return negative_run_pct  
    
def yards_per_rush(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    ypr_avg = 0
    rush_plays = 0
    rush_yards = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            rush_plays += df_mini['home_rush_att'][i]
            rush_yards += df_mini['home_rush_yds'][i]
        elif df_mini['Away_team'][i] == team:
            rush_plays += df_mini['vis_rush_att'][i]
            rush_yards += df_mini['vis_rush_yds'][i]
    try:
        ypr_avg = rush_yards / rush_plays
    except:
        ypr_avg = 0
    return ypr_avg

def yards_per_rush_def(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    ypr_avg = 0
    rush_plays = 0
    rush_yards = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            rush_plays += df_mini['vis_rush_att'][i]
            rush_yards += df_mini['vis_rush_yds'][i]
        elif df_mini['Away_team'][i] == team:
            rush_plays += df_mini['home_rush_att'][i]
            rush_yards += df_mini['home_rush_yds'][i]
    try:
        ypr_avg = rush_yards / rush_plays
    except:
        ypr_avg = 0
    return ypr_avg

def rush_yards_per_game(x):
    team = x[0]
    week = x[1]

    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    total_rush = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            total_rush += df_mini['home_rush_yds'][i]
        elif df_mini['Away_team'][i] == team:
            total_rush += df_mini['vis_rush_yds'][i]
    try:
        rush_yds_per_game = total_rush / len(df_mini)        
    except:
        rush_yds_per_game = 0
    return rush_yds_per_game  

def rush_yards_per_game_def(x):
    team = x[0]
    week = x[1]

    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    total_rush = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            total_rush += df_mini['vis_rush_yds'][i]
        elif df_mini['Away_team'][i] == team:
            total_rush += df_mini['home_rush_yds'][i]
    try:
        rush_yds_per_game = total_rush / len(df_mini)        
    except:
        rush_yds_per_game = 0
    return rush_yds_per_game  

In [515]:
df['home_positive_run_pct'] = df[['Home_team','Week']].apply(positive_run_pct,axis=1)
df['home_positive_run_pct_def'] = df[['Home_team','Week']].apply(positive_run_pct_def,axis=1)
df['away_positive_run_pct'] = df[['Away_team','Week']].apply(positive_run_pct,axis=1)
df['away_positive_run_pct_def'] = df[['Away_team','Week']].apply(positive_run_pct_def,axis=1)

df['home_negative_run_pct'] = df[['Home_team','Week']].apply(negative_run_pct,axis=1)
df['home_negative_run_pct_def'] = df[['Home_team','Week']].apply(negative_run_pct_def,axis=1)
df['away_negative_run_pct'] = df[['Away_team','Week']].apply(negative_run_pct,axis=1)
df['away_negative_run_pct_def'] = df[['Away_team','Week']].apply(negative_run_pct_def,axis=1)

df['home_yds_per_rush'] = df[['Home_team','Week']].apply(yards_per_rush,axis=1)
df['home_yds_per_rush_def'] = df[['Home_team','Week']].apply(yards_per_rush_def,axis=1)
df['away_yds_per_rush'] = df[['Away_team','Week']].apply(yards_per_rush,axis=1)
df['away_yds_per_rush_def'] = df[['Away_team','Week']].apply(yards_per_rush_def,axis=1)

df['home_rush_yds_pg'] = df[['Home_team','Week']].apply(rush_yards_per_game,axis=1)
df['home_rush_yds_pg_def'] = df[['Home_team','Week']].apply(rush_yards_per_game_def,axis=1)
df['away_rush_yds_pg'] = df[['Away_team','Week']].apply(rush_yards_per_game,axis=1)
df['away_rush_yds_pg_def'] = df[['Away_team','Week']].apply(rush_yards_per_game_def,axis=1)

#### General Stats

In [516]:
def third_down_conv_pct(x):
    team = x[0]
    week = x[1]

    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    third_down_att = 0
    third_down_conv = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            third_down_att += df_mini['home_third_down_att'][i]
            third_down_conv += df_mini['home_third_down_conv'][i]
        elif df_mini['Away_team'][i] == team:
            third_down_att += df_mini['vis_third_down_att'][i]
            third_down_conv += df_mini['vis_third_down_conv'][i]
    try:
        third_down_conv_pct = (third_down_conv / third_down_att) * 100 
    except:
        third_down_conv_pct = 0
    return third_down_conv_pct
  
def third_down_conv_pct_def(x):
    team = x[0]
    week = x[1]

    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    third_down_att = 0
    third_down_conv = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            third_down_att += df_mini['vis_third_down_att'][i]
            third_down_conv += df_mini['vis_third_down_conv'][i]
        elif df_mini['Away_team'][i] == team:
            third_down_att += df_mini['home_third_down_att'][i]
            third_down_conv += df_mini['home_third_down_conv'][i]
    try:
        third_down_conv_pct = (third_down_conv / third_down_att) * 100 
    except:
        third_down_conv_pct = 0
    return third_down_conv_pct

def scoring_drive_pct(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    scoring_drives = 0
    total_drives = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            scoring_drives += df_mini['home_scoring_drives'][i]
            total_drives += df_mini['home_total_drives'][i]  
        elif df_mini['Away_team'][i] == team:
            scoring_drives += df_mini['vis_scoring_drives'][i]
            total_drives += df_mini['vis_total_drives'][i] 
    try:
        scoring_drive_pct = (scoring_drives / total_drives) * 100
    except:
        scoring_drive_pct = 0
    return scoring_drive_pct

def scoring_drive_pct_def(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    scoring_drives = 0
    total_drives = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            scoring_drives += df_mini['vis_scoring_drives'][i]
            total_drives += df_mini['vis_total_drives'][i]  
        elif df_mini['Away_team'][i] == team:
            scoring_drives += df_mini['home_scoring_drives'][i]
            total_drives += df_mini['home_total_drives'][i] 
    try:
        scoring_drive_pct = (scoring_drives / total_drives) * 100
    except:
        scoring_drive_pct = 0
    return scoring_drive_pct

def average_time_of_poss(x):
    team = x[0]
    week = x[1]

    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    time_of_poss = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            time_of_poss += df_mini['home_time_of_poss'][i]
        if df_mini['Away_team'][i] == team:
            time_of_poss += df_mini['vis_time_of_poss'][i]
    try:
        avg_time_of_poss = time_of_poss / len(df_mini)
    except:
        avg_time_of_poss = 0
    return avg_time_of_poss

def average_time_of_poss_def(x):
    team = x[0]
    week = x[1]

    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    time_of_poss = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            time_of_poss += df_mini['vis_time_of_poss'][i]
        if df_mini['Away_team'][i] == team:
            time_of_poss += df_mini['home_time_of_poss'][i]
    try:
        avg_time_of_poss = time_of_poss / len(df_mini)
    except:
        avg_time_of_poss = 0
    return avg_time_of_poss

def winning_play_pct(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    plays_winning = 0
    total_plays = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            plays_winning += df_mini['home_plays_winning'][i]
            total_plays += df_mini['total_plays'][i]
        if df_mini['Away_team'][i] == team:
            plays_winning += df_mini['vis_plays_winning'][i]
            total_plays += df_mini['total_plays'][i]
    try:
        winning_play_pct = (plays_winning/total_plays)*100
    except:
        winning_play_pct = 0
    return winning_play_pct

In [517]:
df['home_third_down_conv_pct'] = df[['Home_team','Week']].apply(third_down_conv_pct,axis=1)
df['home_third_down_conv_pct_def'] = df[['Home_team','Week']].apply(third_down_conv_pct_def,axis=1)
df['away_third_down_conv_pct'] = df[['Away_team','Week']].apply(third_down_conv_pct,axis=1)
df['away_third_down_conv_pct_def'] = df[['Away_team','Week']].apply(third_down_conv_pct_def,axis=1)

df['home_scoring_drive_pct'] = df[['Home_team','Week']].apply(scoring_drive_pct,axis=1)
df['home_scoring_drive_pct_def'] = df[['Home_team','Week']].apply(scoring_drive_pct_def,axis=1)
df['away_scoring_drive_pct'] = df[['Away_team','Week']].apply(scoring_drive_pct,axis=1)
df['away_scoring_drive_pct_def'] = df[['Away_team','Week']].apply(scoring_drive_pct_def,axis=1)

df['home_avg_time_of_poss'] = df[['Home_team','Week']].apply(average_time_of_poss,axis=1)
df['home_avg_time_of_poss_def'] = df[['Home_team','Week']].apply(average_time_of_poss_def,axis=1)
df['away_avg_time_of_poss'] = df[['Away_team','Week']].apply(average_time_of_poss,axis=1)
df['away_avg_time_of_poss_def'] = df[['Away_team','Week']].apply(average_time_of_poss_def,axis=1)

df['home_winning_play_pct'] = df[['Home_team','Week']].apply(winning_play_pct,axis=1)
df['vis_winning_play_pct'] = df[['Away_team','Week']].apply(winning_play_pct,axis=1)

#### Margin of Victory

In [518]:
def avg_winloss_margin(x):
    team = x[0]
    week = x[1]

    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    winloss_margin = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            winloss_margin += df_mini['Home_margin_of_win/loss'][i]
        if df_mini['Away_team'][i] == team:
            winloss_margin += df_mini['Away_margin_of_win/loss'][i]
    try:
        avg_winloss_margin = winloss_margin / len(df_mini)
    except:
        avg_winloss_margin = 0
    return avg_winloss_margin

In [519]:
df['home_avg_winloss_margin'] = df[['Home_team','Week']].apply(avg_winloss_margin,axis=1)
df['away_avg_winloss_margin'] = df[['Away_team','Week']].apply(avg_winloss_margin,axis=1)

### Adjust Columns for Vegas Line, Over Under, Margin of Victory, and Cover

In [520]:
df[['Favorite','Vegas_line']] = df['Vegas_line'].str.split('-',expand=True)
df['Vegas_line'] = df['Vegas_line'].astype(float)
df[['Point_total','Over_under']] = df['Over_under'].str.split(' ',expand=True)
df['Over_under'] = np.where(df.Over_under == '(over)',1,0)
df['Point_total'] = df['Point_total'].astype(float)

In [521]:
df['Favorite'] = df['Favorite'].str.replace('Arizona Cardinals','ARI')
df['Favorite'] = df['Favorite'].str.replace('Atlanta Falcons','ATL')
df['Favorite'] = df['Favorite'].str.replace('Baltimore Ravens','BAL')
df['Favorite'] = df['Favorite'].str.replace('Buffalo Bills','BUF')
df['Favorite'] = df['Favorite'].str.replace('Carolina Panthers','CAR')
df['Favorite'] = df['Favorite'].str.replace('Chicago Bears','CHI')
df['Favorite'] = df['Favorite'].str.replace('Cincinnati Bengals','CIN')
df['Favorite'] = df['Favorite'].str.replace('Cleveland Browns','CLE')
df['Favorite'] = df['Favorite'].str.replace('Dallas Cowboys','DAL')
df['Favorite'] = df['Favorite'].str.replace('Denver Broncos','DEN')
df['Favorite'] = df['Favorite'].str.replace('Detroit Lions','DET')
df['Favorite'] = df['Favorite'].str.replace('Green Bay Packers','GNB')
df['Favorite'] = df['Favorite'].str.replace('Houston Texans','HOU')
df['Favorite'] = df['Favorite'].str.replace('Indianapolis Colts','IND')
df['Favorite'] = df['Favorite'].str.replace('Jacksonville Jaguars','JAX')
df['Favorite'] = df['Favorite'].str.replace('Kansas City Chiefs','KAN')
df['Favorite'] = df['Favorite'].str.replace('Los Angeles Chargers','LAC')
df['Favorite'] = df['Favorite'].str.replace('Los Angeles Rams','LAR')
df['Favorite'] = df['Favorite'].str.replace('Miami Dolphins','MIA')
df['Favorite'] = df['Favorite'].str.replace('Minnesota Vikings','MIN')
df['Favorite'] = df['Favorite'].str.replace('New Orleans Saints','NOR')
df['Favorite'] = df['Favorite'].str.replace('New England Patriots','NWE')
df['Favorite'] = df['Favorite'].str.replace('New York Giants','NYG')
df['Favorite'] = df['Favorite'].str.replace('New York Jets','NYJ')
df['Favorite'] = df['Favorite'].str.replace('Oakland Raiders','OAK')
df['Favorite'] = df['Favorite'].str.replace('Philadelphia Eagles','PHI')
df['Favorite'] = df['Favorite'].str.replace('Pittsburgh Steelers','PIT')
df['Favorite'] = df['Favorite'].str.replace('Seattle Seahawks','SEA')
df['Favorite'] = df['Favorite'].str.replace('San Francisco 49ers','SFO')
df['Favorite'] = df['Favorite'].str.replace('Tampa Bay Buccaneers','TAM')
df['Favorite'] = df['Favorite'].str.replace('Tennessee Titans','TEN')
df['Favorite'] = df['Favorite'].str.replace('Washington Redskins','WAS')

In [522]:
df.Home_team = df.Home_team.astype(str)
df.Home_team = df.Home_team.apply(lambda x: x.strip())
df.Away_team = df.Away_team.astype(str)
df.Away_team = df.Away_team.apply(lambda x: x.strip())
df.Favorite = df.Favorite.astype(str)
df.Favorite = df.Favorite.apply(lambda x: x.strip())

df['Home_line'] = np.where((df.Home_team == df.Favorite),df.Vegas_line * -1, df.Vegas_line)
df['Home_cover'] = np.where((df['Home_margin_of_win/loss'] + df['Home_line']) > 0 , 1, 0)
df['Away_line'] = np.where((df.Away_team == df.Favorite),df.Vegas_line * -1, df.Vegas_line)
df['Away_cover'] = np.where((df['Away_margin_of_win/loss'] + df['Away_line']) > 0 , 1, 0)
df['Home_win'] = np.where(df.Home_points > df.Away_points,1,0)
df['Away_win'] = np.where(df.Away_points > df.Home_points,1,0)

#### Calculate Averages

In [545]:
def pct_games_covered(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    games_covered = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            games_covered += df_mini['Home_cover'][i]
        if df_mini['Away_team'][i] == team:
            games_covered += df_mini['Away_cover'][i]
    try:
        pct_games_covered = (float(games_covered)/len(df_mini))*100
    except:
        pct_games_covered = 0
    return pct_games_covered

def pct_games_won(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    games_won = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            games_won += df_mini['Home_win'][i]
        if df_mini['Away_team'][i] == team:
            games_won += df_mini['Away_win'][i]
    try:
        pct_games_won = (float(games_won)/len(df_mini))*100
    except:
        pct_games_won = 0
    return pct_games_won 

def pct_games_over(x):
    team = x[0]
    week = x[1]
    
    df_mini = df[((df['Home_team']==team) | (df[('Away_team')]==team)) & (df[('Week')]<week)]
    df_mini.reset_index(drop=True,inplace=True)
    
    games_over = 0
    for i in range(len(df_mini)):
        if df_mini['Home_team'][i] == team:
            games_over += df_mini['Over_under'][i]
        if df_mini['Away_team'][i] == team:
            games_over += df_mini['Over_under'][i]
    try:
        pct_games_over = (float(games_over)/len(df_mini))*100
    except:
        pct_games_over = 0
    return pct_games_over

In [546]:
df['home_pct_games_covered']  = df[['Home_team','Week']].apply(pct_games_covered,axis=1)
df['away_pct_games_covered']  = df[['Away_team','Week']].apply(pct_games_covered,axis=1)
df['home_pct_games_won']      = df[['Home_team','Week']].apply(pct_games_won,axis=1)
df['away_pct_games_won']      = df[['Away_team','Week']].apply(pct_games_won,axis=1)
df['home_pct_games_over']     = df[['Home_team','Week']].apply(pct_games_over,axis=1)
df['away_pct_games_over']     = df[['Away_team','Week']].apply(pct_games_over,axis=1)

In [547]:
df.to_csv('season_computed_2017.csv')